Cleaning strategy for NPPES file:
- read in only columns relevant to this project
- process taxonomy code columns to determine primary
- drop taxonomy code columns (keep only added primary code column)
- join classification crosswalk to get taxonomy name
- join CBSA crosswalk to match provider on zip code
- make column names better for SQL
-put into SQLite

Nashville CBSA: 34980

In [4]:
import pandas as pd

In [6]:
df1 = pd.read_csv("../data/DocGraph_Hop_Teaming_2018.csv")
df1.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880


In [9]:
filtered = df1.filter((pd.col('transaction_count') > 50) & (pd.col('average_day_wait') > 50))
filtered.shape

AttributeError: module 'pandas' has no attribute 'col'

In [7]:
filtered.shape

NameError: name 'filtered' is not defined

In [ ]:
df2 = pd.read_csv("../data/nucc_taxonomy_240.csv")
df2.head()

In [6]:
df3 = pd.read_csv("../data/NPPES_Data_Dissemination_February_2024/npidata_pfile_20050523-20240211.csv", nrows = 10000, low_memory= False)
df3.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1679576722,1.0,NaN,NaN,NaN,WIEBE,DAVID,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,NaN,NaN,PILCHER,WILLIAM,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,NaN,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df3.columns

Index(['NPI', 'Entity Type Code', 'Replacement NPI',
       'Employer Identification Number (EIN)',
       'Provider Organization Name (Legal Business Name)',
       'Provider Last Name (Legal Name)', 'Provider First Name',
       'Provider Middle Name', 'Provider Name Prefix Text',
       'Provider Name Suffix Text',
       ...
       'Healthcare Provider Taxonomy Group_7',
       'Healthcare Provider Taxonomy Group_8',
       'Healthcare Provider Taxonomy Group_9',
       'Healthcare Provider Taxonomy Group_10',
       'Healthcare Provider Taxonomy Group_11',
       'Healthcare Provider Taxonomy Group_12',
       'Healthcare Provider Taxonomy Group_13',
       'Healthcare Provider Taxonomy Group_14',
       'Healthcare Provider Taxonomy Group_15', 'Certification Date'],
      dtype='object', length=330)

In [7]:
#df3.columns = [x.lower().replace(' ', '_') for x in df3.columns]
#print(pd.io.sql.get_schema(df3, 'nppes'))

CREATE TABLE "nppes" (
"npi" INTEGER,
  "entity_type_code" REAL,
  "replacement_npi" REAL,
  "employer_identification_number_(ein)" TEXT,
  "provider_organization_name_(legal_business_name)" TEXT,
  "provider_last_name_(legal_name)" TEXT,
  "provider_first_name" TEXT,
  "provider_middle_name" TEXT,
  "provider_name_prefix_text" TEXT,
  "provider_name_suffix_text" TEXT,
  "provider_credential_text" TEXT,
  "provider_other_organization_name" TEXT,
  "provider_other_organization_name_type_code" REAL,
  "provider_other_last_name" TEXT,
  "provider_other_first_name" TEXT,
  "provider_other_middle_name" TEXT,
  "provider_other_name_prefix_text" TEXT,
  "provider_other_name_suffix_text" TEXT,
  "provider_other_credential_text" TEXT,
  "provider_other_last_name_type_code" REAL,
  "provider_first_line_business_mailing_address" TEXT,
  "provider_second_line_business_mailing_address" TEXT,
  "provider_business_mailing_address_city_name" TEXT,
  "provider_business_mailing_address_state_name" TEXT,

In [8]:
df4 = pd.read_csv("../data/NPPES_Data_Dissemination_February_2024/npidata_pfile_20050523-20240211.csv", nrows = 100)
df4.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1679576722,1.0,NaN,NaN,NaN,WIEBE,DAVID,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,NaN,NaN,PILCHER,WILLIAM,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,NaN,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
